# 필요한 모듈 임포트

In [49]:
pip install selenium==4.8.2

In [50]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
!pip install webdriver_manager
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [87]:
import re

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
codi_list = []
url = 'https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)

columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
values = []
soup = BeautifulSoup(driver.page_source, 'html.parser')
data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

for i, row in enumerate(data_rows):

    print('{}번째 코디 정보 크롤링'.format(i+1))
    blank = []

    codimap_category = row.find('span', attrs={'class':'style-list-information__text'})
    if codimap_category:
        codimap_category = codimap_category.get_text().strip()
        blank.append(codimap_category)
    else:
        blank.append('Something is wrong')
        print('{}번째 codimap_category 가져올 때 문제 발생'.format(i+1))
        continue

    codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})
    if codimap_title:
        codimap_title = codimap_title.get_text().strip()
        blank.append(codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 codimap_title 가져올 때 문제 발생'.format(i+1))
        continue

    post_information_texts = row.find_all('span', attrs={'class': 'post-information__text'})
    if len(post_information_texts) >= 3:
        codimap_date = post_information_texts[0].get_text().strip()
        views_text = post_information_texts[1].get_text().strip()
        comment_numbers_text = post_information_texts[2].get_text().strip()

        views = re.sub('[^0-9]', '', views_text)  # 숫자 이외의 문자 제거
        comment_numbers = re.sub('[^0-9]', '', comment_numbers_text)  # 숫자 이외의 문자 제거

        blank.append(codimap_date)
        blank.append(views)
        blank.append(comment_numbers)
    else:
        blank.extend(['Something is wrong'] * 3)
        print('{}번째 post-information__text 가져올 때 문제 발생'.format(i+1))
        continue

    values.append(blank)
    print('---------------------------------------------------')

df = pd.DataFrame(values, columns=columns)
codi_list.append(df)


1번째 코디 정보 크롤링
---------------------------------------------------
2번째 코디 정보 크롤링
---------------------------------------------------
3번째 코디 정보 크롤링
---------------------------------------------------
4번째 코디 정보 크롤링
---------------------------------------------------
5번째 코디 정보 크롤링
---------------------------------------------------
6번째 코디 정보 크롤링
---------------------------------------------------
7번째 코디 정보 크롤링
---------------------------------------------------
8번째 코디 정보 크롤링
---------------------------------------------------
9번째 코디 정보 크롤링
---------------------------------------------------
10번째 코디 정보 크롤링
---------------------------------------------------
11번째 코디 정보 크롤링
---------------------------------------------------
12번째 코디 정보 크롤링
---------------------------------------------------
13번째 코디 정보 크롤링
---------------------------------------------------
14번째 코디 정보 크롤링
---------------------------------------------------
15번째 코디 정보 크롤링
---------------------------------------------------
16번째

In [88]:
df.head(5)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,15972,49
1,캐주얼,블랙 매니아,21.02.23,34641,46
2,캐주얼,오늘의 코디,22.03.22,12172,38
3,댄디,추천 댄디 룩,21.09.15,20162,27
4,캐주얼,릴랙스 룩,21.02.23,23514,22


In [90]:
df.shape

(60, 5)

# Dataframe을 csv 파일로 저장

In [91]:
df.to_csv('0707_크롤링_과제.csv', encoding = 'utf-8-sig')